# Link Prediction

We take the embedded data and calculate the cosine similarity between nodes. Cosine similarity is equal to normalized dot product similarity (source: https://zhang-yang.medium.com/cosine-similarity-dot-product-for-normalized-vectors-c07bdb61c9d1). If nodes are similar, but not linked yet, we predict a link between them.

The links are predicted on the partial network (all nodes, but 80% of the links are randomly removed) and tested on the ground truth full network.

## Imports

In [7]:
import itertools
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
# Open link csv
all_links = np.genfromtxt('../preprocessing/preprocessed_data/all_links.csv', delimiter=',', dtype=int)
removed_links = np.genfromtxt('../preprocessing/preprocessed_data/removed_links.csv', delimiter=',', dtype=int)

highest_node_id = all_links.max()
all_links

array([[   0,    1],
       [   0,    2],
       [   0,    3],
       ...,
       [4027, 4032],
       [4027, 4038],
       [4031, 4038]])

## Functions

In [9]:
def predict_links(df):
    # Initialize empty predicted links
    predicted_links = []

    # Iterate over all nodes and compare them to nodes they are not yet linked to
    # Predicts link when cosine similarity > 97.5%
    for node in range(0, highest_node_id + 1):
        # Slices of dataframes containing the node itself
        # and the nodes it is not connected to
        node_df = df[df.index == node]
        unconnected_nodes = [n for n in range(0, highest_node_id + 1) 
                             if n not in removed_links[removed_links[:,0] == node][:,1] +
                             [node]]
        unconnected_nodes_df = df[df.index.isin(unconnected_nodes)]

        # Cosine similarity between the node itself
        # and the nodes it is not already connected to
        similarity_scores = cosine_similarity(node_df, unconnected_nodes_df)[0].tolist()
        similarity_dict = dict(zip(unconnected_nodes, similarity_scores))
            
        # If similarity > 97.5%, append to the list of predictions
        for pair in similarity_dict.items():
            if pair[1] > 0.975:
                predicted_links.append((node, pair[0]))

    return predicted_links

In [10]:
def link_array_to_pairs(link_array):
    pairs = []

    # Add a bidirectional pair for each link
    # E.g. (1,0) and (0,1)
    for link in link_array:
        pairs.append((link[0], link[1]))
        pairs.append((link[1], link[0]))

    return pairs

In [11]:
def metrics(pred):
    # Convert the arrays to bidirectional lists of pairs for easier comparison
    all_link_pairs = link_array_to_pairs(all_links)
    removed_link_pairs = link_array_to_pairs(removed_links)

    # False positive predictions
    # i.e. it predicted a link that isn't there
    fp = set(pred) - set(all_link_pairs)

    # False negative predictions
    # i.e. it didnt predict a link that should have been there
    fn = set(all_link_pairs) - set(pred) - set(removed_link_pairs)

    # True positive predictions
    # i.e. all correct predictions
    tp = set(pred) - fp

    # True negative predictions
    # i.e. all links that were correctly not predicted
    all_possible_pairs = itertools.product(list(range(0, highest_node_id + 1)), list(range(0, highest_node_id + 1)))
    tn = set(all_possible_pairs) - set(all_link_pairs) - set(pred)
    
    total = len(tp) + len(tn) + len(fp) + len(fn)
    
    # Accuracy
    accuracy = (len(tp) + len(tn))/total
    
    # Precision
    precision = len(tp) / (len(tp) + len(fp))
    
    # Make confusion matrix pairs with (absolute values, percentages)
    fp = (len(fp), len(fp)/total)
    fn = (len(fn), len(fn)/total)
    tp = (len(tp), len(tp)/total)
    tn = (len(tn), len(tn)/total)
    
    return fp, fn, tp, tn, accuracy, precision

## Predict for Node2Vec

This takes a while to run...

In [12]:
dimensions = [8, 16, 32, 64, 128]

for dim in dimensions:
      # Import and make predictions
      print('Predicting links for dimension: ', str(dim), '...')
      node2vec = pd.read_csv('../node_embeddings/embedded_partial_data/node2vec_' + str(dim) + '.csv')
      node2vec_pred = predict_links(node2vec)

      # Obtain and print metrics
      fp, fn, tp, tn, accuracy, precision = metrics(node2vec_pred)
      print('Node2vec dim: ', str(dim), '\n',
            'fp: ', fp, '\n',
            'fn: ', fn, '\n',
            'tp: ', tp, '\n',
            'tn: ', tn, '\n',
            'accuracy: ', accuracy, '\n',
            'precision: ', precision)

Predicting links for dimension:  8 ...
Node2vec dim:  8 
 fp:  (73777, 0.004538777085121649) 
 fn:  (16352, 0.0010059785962550551) 
 tp:  (101414, 0.006239011335653753) 
 tn:  (16063276, 0.9882162329829696) 
 accuracy:  0.9944552443186233 
 precision:  0.5788767687837846
Predicting links for dimension:  16 ...
Node2vec dim:  16 
 fp:  (12849, 0.0007930700650152037) 
 fn:  (29107, 0.0017965515123665294) 
 tp:  (35435, 0.0021871303411793715) 
 tn:  (16124204, 0.9952232480814389) 
 accuracy:  0.9974103784226183 
 precision:  0.7338870019053931
Predicting links for dimension:  32 ...
Node2vec dim:  32 
 fp:  (4988, 0.00030836252205923353) 
 fn:  (34405, 0.0021269471875396814) 
 tp:  (4308, 0.00026632432739197634) 
 tn:  (16132065, 0.9972983659630091) 
 accuracy:  0.9975646902904011 
 precision:  0.46342512908777966
Predicting links for dimension:  64 ...
Node2vec dim:  64 
 fp:  (4445, 0.00027483808203456107) 
 fn:  (34810, 0.002152331526574369) 
 tp:  (1296, 8.013276812526235e-05) 
 tn:  

## Predict for Splitter